# Exercício 2


## Sobre a base de dados

Neste laboratório você utilizará um dataset contendo informações sobre filmes e cinemas. 

**Collections:**
* comments
* movies
* sessions
* theaters
* users




Veja um exemplo de documento de um dos filmes da collection `movies` e entenda a estrutura/schema:

```python
{
	'_id': ObjectId('573a1390f29313caabcd4135'),
	'awards': {
		'nominations': 0,
		'text': '1 win.',
		'wins': 1
	},
	'cast': ['Charles Kayser', 'John Ott'],
	'countries': ['USA'],
	'directors': ['William K.L. Dickson'],
	'fullplot': 'A stationary camera looks at a large ...',
	'genres': ['Short'],
	'imdb': {
		'id': 5,
		'rating': 6.2,
		'votes': 1189
	},
	'lastupdated': '2015-08-26 00:03:50.133000000',
	'num_mflix_comments': 1,
	'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
	'rated': 'UNRATED',
	'released': datetime(1893, 5, 9, 0, 0),
	'runtime': 1,
	'title': 'Blacksmith Scene',
	'tomatoes': {
		'lastUpdated': datetime(2015, 6, 28, 18, 34, 9),
		'viewer': {
			'meter': 32,
			'numReviews': 184,
			'rating': 3.0
		},
	},
	'type': 'movie',
	'year': 1893,
}
```



## Usando Python

In [8]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
warnings.filterwarnings('ignore')
mongoclient = MongoClient('localhost', 27017)
db = mongoclient.mflix
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mflix')


In [2]:
db.movies.find({}).count()

23539

## Questões Exemplo

### [Questão Exemplo A]  Mostre um documento qualquer da coleção movies

In [3]:
result = db.movies.find({}).limit(1)

pprint( list(result) )

[
    {
        '_id': ObjectId('573a1390f29313caabcd4135'),
        'awards': {'nominations': 0, 'text': '1 win.', 'wins': 1},
        'cast': ['Charles Kayser', 'John Ott'],
        'countries': ['USA'],
        'directors': ['William K.L. Dickson'],
        'fullplot': 'A stationary camera looks at a large anvil with a blacksmith behind it and one on either side. The smith in the middle draws a heated metal rod from the fire, places it on the anvil, and all three begin a rhythmic hammering. After several blows, the metal goes back in the fire. One smith pulls out a bottle of beer, and they each take a swig. Then, out comes the glowing metal and the hammering resumes.',
        'genres': ['Short'],
        'imdb': {'id': 5, 'rating': 6.2, 'votes': 1189},
        'lastupdated': '2015-08-26 00:03:50.133000000',
        'num_mflix_comments': 1,
        'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
        'rated': 'UNRATED',
        'released': datetime.datet

### [Questão Exemplo B]  Encontre os diretores do filme "Inglourious Basterds"

In [4]:
result = db.movies.find({"title": "Inglourious Basterds"}, {"_id":0,"directors": 1})

list(result)

[{'directors': ['Quentin Tarantino', 'Eli Roth']}]

### [Questão 1] Mostre o ano de lançamento de todos os filme que começam com "One Night"

In [5]:
result = db.movies.find({'title':{'$regex':'^One Night'}},
                        {'title':1,'year':1, '_id':0}
                       )
list(result)

[{'title': 'One Night of Love', 'year': 1934},
 {'title': 'One Night Stand', 'year': 1997},
 {'title': 'One Night with the King', 'year': 2006},
 {'title': 'One Night in City', 'year': 2007}]

### [Questão 2] Quantos filmes o diretor Quentin Tarantino dirigiu?

<!--
query = { 
         "directors": "Quentin Tarantino"
}

projection = { 
               "title": 1,
               "_id": 0   
}


result = db.movies.find(query, projection).distinct("title")
list(result)
-->

In [7]:
result = db.movies.find({'directors':'Quentin Tarantino'}).count()

print(result)

14


### [Questão 3] Quantos filmes do gênero "Drama" o diretor Quentin Tarantino dirigiu entre 1990 e 2005?


<!--
query = { 
         "directors": "Quentin Tarantino",
         "genres" : "Drama",
           "$and" : [ 
                     { "year": {"$gte":1990}},
                     { "year": {"$gte":1990}}
                    ]
}

projection = { 
               "title": 1,
               "genres":1, 
               "year": 1,
               "_id": 0
}


result = db.movies.find(query, projection)
list(result)
-->

In [57]:
result = db.movies.find({ 'year': { '$gte': 1990, '$lte': 2005},
                         'directors':'Quentin Tarantino',
                         'genres':'Drama',},
                        {'_id':0,'year':1,'title':1})


list(result)

[{'title': 'Reservoir Dogs', 'year': 1992},
 {'year': 1994, 'title': 'Pulp Fiction'},
 {'year': 1997, 'title': 'Jackie Brown'}]

### [Questão 4] Qual filme do diretor Quentin Tarantino teve mais prêmios?

Dica: Sort em Pymongo https://www.w3schools.com/python/python_mongodb_sort.asp

<!-- 
query = { 
         "directors": "Quentin Tarantino"
}

projection = { 
               "title": 1,
               "awards.wins":1,
               "_id": 0   
}


result = db.movies.find(query, projection).sort("awards.wins",-1)
list(result)
-->

In [48]:
result = db.movies.find({'directors':'Quentin Tarantino'},{'_id': 0 ,'title':1}).sort('awards.wins',-1).limit(1)

list(result)

[{'title': 'Inglourious Basterds'}]

### [Questão Extra] Quais documentos (filmes) possuem mesmo nome e quantas vezes cada um se repete?

<!--

-->

In [20]:
result = db.movies.aggregate([
    {"$group" : {
        '_id':"$title",
        'count':{'$sum':1}
     }
    },{
        '$match': {
            'count': {'$gt': 1}
        }
    }])

list(result)

[{'_id': 'Django Unchained', 'count': 2},
 {'_id': 'Twist', 'count': 2},
 {'_id': 'The Nun', 'count': 4},
 {'_id': 'Quo Vadis', 'count': 2},
 {'_id': 'Chaos', 'count': 2},
 {'_id': "Boys Don't Cry", 'count': 2},
 {'_id': 'Joanna', 'count': 2},
 {'_id': 'Alice', 'count': 4},
 {'_id': 'Stella', 'count': 3},
 {'_id': 'Underground: The Julian Assange Story', 'count': 2},
 {'_id': 'Boy Meets Girl', 'count': 3},
 {'_id': 'State of Play', 'count': 2},
 {'_id': 'Posse', 'count': 2},
 {'_id': 'Police', 'count': 2},
 {'_id': 'Turbulence 2: Fear of Flying', 'count': 2},
 {'_id': 'Lola', 'count': 2},
 {'_id': 'Undertow', 'count': 2},
 {'_id': 'Children of Heaven', 'count': 2},
 {'_id': 'The Chumscrubber', 'count': 2},
 {'_id': 'Hei an zhi guang', 'count': 2},
 {'_id': 'The Producers', 'count': 2},
 {'_id': 'The Best Years of Our Lives', 'count': 2},
 {'_id': 'Houston', 'count': 2},
 {'_id': 'Deck the Halls', 'count': 2},
 {'_id': 'Bloody Sunday', 'count': 2},
 {'_id': 'Born to Be Wild', 'count': 2

### [Questão Extra] Quantos filmes cada diretor participou da direção? (ignore filmes repetidos)

In [55]:
result = db.movies.aggregate([
    {"$group" : {
        '_id':"$directors",
        'title': {'$addToSet': "$title"},
     }
    },
    {
        '$unwind':"$title"
    },
    {
        '$group': {'_id': "$_id", 'count': { '$sum':1} }
    }])

list(result)



[{'_id': ['Debra Granik'], 'count': 3},
 {'_id': ['Kristin Hanggi'], 'count': 1},
 {'_id': ["John 'Bud' Cardos"], 'count': 1},
 {'_id': ['Nikhil Advani'], 'count': 3},
 {'_id': ['Jim Capobianco'], 'count': 1},
 {'_id': ['Ron Satlof'], 'count': 2},
 {'_id': ['Don Hardy Jr.', 'Dana Nachman'], 'count': 1},
 {'_id': ['Tom Browne'], 'count': 1},
 {'_id': ['Mark Helenowski', 'Kevin Pang'], 'count': 1},
 {'_id': ['Bodil Ipsen', 'Lau Lauritzen'], 'count': 1},
 {'_id': ['Michael Tiddes'], 'count': 1},
 {'_id': ['Shivajee Chandrabhushan'], 'count': 1},
 {'_id': ['Charles Nemes'], 'count': 1},
 {'_id': ['Jaime Maestro'], 'count': 1},
 {'_id': ['Arthur Hiller'], 'count': 15},
 {'_id': ['Maud Nycander'], 'count': 1},
 {'_id': ['Penny Marshall'], 'count': 5},
 {'_id': ['Marèa Luisa Bemberg'], 'count': 3},
 {'_id': ['Joseph B. Vasquez'], 'count': 1},
 {'_id': ['Rupert Goold'], 'count': 1},
 {'_id': ['Hanne Larsen'], 'count': 1},
 {'_id': ['Dean Parisot'], 'count': 3},
 {'_id': ['Salim Akil'], 'count'

### [Questão Extra] Qual a somatória de prêmios recebidos pelos filmes para cada diretor? (ignore filmes repetidos)

In [57]:
result = db.movies.aggregate([
    {"$group" : {
        '_id':"$directors",
        'title': {'$addToSet': "$title"},
     }
    },
    {
        '$unwind':"$title"
    },
    {
        '$group': {
            '_id': "$_id",
            'awards': { '$sum': '$awards.wins'}
        }
    }])


list(result)

[{'_id': ['Debra Granik'], 'awards': 0},
 {'_id': ['Kristin Hanggi'], 'awards': 0},
 {'_id': ["John 'Bud' Cardos"], 'awards': 0},
 {'_id': ['Nikhil Advani'], 'awards': 0},
 {'_id': ['Jim Capobianco'], 'awards': 0},
 {'_id': ['Ron Satlof'], 'awards': 0},
 {'_id': ['Don Hardy Jr.', 'Dana Nachman'], 'awards': 0},
 {'_id': ['Tom Browne'], 'awards': 0},
 {'_id': ['Mark Helenowski', 'Kevin Pang'], 'awards': 0},
 {'_id': ['Bodil Ipsen', 'Lau Lauritzen'], 'awards': 0},
 {'_id': ['Michael Tiddes'], 'awards': 0},
 {'_id': ['Shivajee Chandrabhushan'], 'awards': 0},
 {'_id': ['Charles Nemes'], 'awards': 0},
 {'_id': ['Jaime Maestro'], 'awards': 0},
 {'_id': ['Arthur Hiller'], 'awards': 0},
 {'_id': ['Maud Nycander'], 'awards': 0},
 {'_id': ['Penny Marshall'], 'awards': 0},
 {'_id': ['Marèa Luisa Bemberg'], 'awards': 0},
 {'_id': ['Joseph B. Vasquez'], 'awards': 0},
 {'_id': ['Rupert Goold'], 'awards': 0},
 {'_id': ['Hanne Larsen'], 'awards': 0},
 {'_id': ['Dean Parisot'], 'awards': 0},
 {'_id': ['